In [ ]:
from google.colab import auth, files
import pandas as pd
from google.cloud import storage
import os
from datetime import datetime
import pytz
import numpy as np

# Authenticate to GCS
auth.authenticate_user()

# Define the GCS bucket
bucket_name = 'prd-marketshare'

# Set up GCS client
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Function to get the latest file path from a bucket folder
def get_latest_file_path(bucket, folder_path):
    blobs = list(bucket.list_blobs(prefix=folder_path))
    if not blobs:
        return None
    latest_blob = max(blobs, key=lambda b: b.time_created)
    return f"gs://{bucket.name}/{latest_blob.name}"

# Function to save files to bucket folders
def save_to_bucket(df, bucket, destination_folder, file_name):
    # Temporary saving the file locally
    local_file_path = f'/tmp/{file_name}'
    df.to_csv(local_file_path, index=False)

    # Upload the file to the specific folder in GCS
    blob = bucket.blob(f'{destination_folder}/{file_name}')
    blob.upload_from_filename(local_file_path)

    # Download the file (if needed)
    #files.download(local_file_path)

# Get the paths of the latest files from specified folders
sd_file_gcs_path = get_latest_file_path(bucket, '04_dictionary/01_emot/02_emot_cleaned/')
pihs_vl_ps_file_gcs_path = get_latest_file_path(bucket, '03_power_schedule/03_pihs_vl_ps_merged/')

# Read the latest files into DataFrames directly from GCS paths
# Read the latest files into DataFrames directly from GCS paths
if sd_file_gcs_path:
    sd = pd.read_csv(sd_file_gcs_path, dtype={'mnemonic_vehicle_id': str})
    sd['mnemonic_vehicle_id'] = sd['mnemonic_vehicle_id'].replace({np.nan: None})
    sd = sd[sd['change_tracker'] != 'removed']
else:
    print("No sd file found")

if pihs_vl_ps_file_gcs_path:
    pihs_vl_ps = pd.read_csv(pihs_vl_ps_file_gcs_path, dtype={'mnemonic_vehicle_id': str})
    pihs_vl_ps['mnemonic_vehicle_id'] = pihs_vl_ps['mnemonic_vehicle_id'].replace({np.nan: None})
else:
    print("No pihs_vl_ps file found")



# Define columns for exact and range matching
exact_match_columns = [
    'sales_group', 'vehicle_design_parent', 'vehicle_platform', 'global_nameplate',
    'electrification', 'propulsion_system_subdesign_architecture', 'ap_px_definition',
    'drive_type', 'transmission_design', 'transmission_program', 'global_sales_sub_segment',
    'vehicle_manufacturer', 'production_brand', 'vehicle_ihs_region', 'mnemonic_vehicle_id'
]

range_match_columns = [
    ('system_voltage_v_lower_bound', 'system_voltage_v_upper_bound', 'system_voltage_v_range_type', 'system_voltage_v'),
    ('ap_system_power_kw_lower_bound', 'ap_system_power_kw_upper_bound', 'ap_system_power_kw_range_type', 'ap_system_power_kw'),
    ('electric_motor_power_kw_lower_bound', 'electric_motor_power_kw_upper_bound', 'electric_motor_power_kw_range_type', 'electric_motor_power_kw'),
    ('power_emotor2_lower_bound',	'power_emotor2_upper_bound',	'power_emotor2_range_type', 'power_none_emotor_2'),
    ('vehicle_sop_start_of_production_lower_bound',	'vehicle_sop_start_of_production_upper_bound',	'vehicle_sop_start_of_production_range_type', 'vehicle_sop_start_of_production')
]

# Convert string columns to lowercase for case-insensitive matching
#for col in exact_match_columns:
#    if sd[col].dtype == 'object':
#        sd[col] = sd[col].str.lower()
#    if pihs_vl_ps[col].dtype == 'object':
#       pihs_vl_ps[col] = pihs_vl_ps[col].str.lower()

# Explicitly set the specified columns in sd to float
float_columns_sd = [
    'system_voltage_v_lower_bound', 'system_voltage_v_upper_bound',
    'ap_system_power_kw_lower_bound', 'ap_system_power_kw_upper_bound',
    'electric_motor_power_kw_lower_bound', 'electric_motor_power_kw_upper_bound',
    'power_emotor2_lower_bound',	'power_emotor2_upper_bound'
]

for col in float_columns_sd:
    sd[col] = pd.to_numeric(sd[col], errors='coerce').astype('float')

# Explicitly set the specified columns in pihs_vl_ps to float
float_columns_pihs_vl_ps = [
    'system_voltage_v',
    'ap_system_power_kw',
    'electric_motor_power_kw',
    'power_none_emotor_2'

]

for col in float_columns_pihs_vl_ps:
    pihs_vl_ps[col] = pd.to_numeric(pihs_vl_ps[col], errors='coerce').astype('float')

# Sort sd by the number of non-null values in descending order

## Extracting lower and upper bounds, and range type column names from range_match_columns
range_bound_columns = [col for rmc in range_match_columns for col in rmc[:3]]  # First three elements in each tuple

## Combine with exact_match_columns
relevant_columns = exact_match_columns + range_bound_columns

## Count non-null values in relevant columns only
non_null_counts = sd[relevant_columns].notnull().sum(axis=1)

## Sort sd by the non-null counts in descending order
sd = sd.loc[non_null_counts.sort_values(ascending=False).index]

sorted_sd = sd

def handle_range_match(sd_row, lower_bound_col, upper_bound_col, range_type_col, pihs_vl_ps_col):
    lower_bound_value = sd_row[lower_bound_col]
    upper_bound_value = sd_row[upper_bound_col]
    range_type = sd_row[range_type_col]

    # Function to format value based on its type (date or numeric)
    def format_value_for_query(value, is_date):
        if pd.notnull(value):
            if is_date:
                try:
                    # Convert the value to datetime and format as a string in 'YYYY-MM-DD' format
                    date_val = pd.to_datetime(value).strftime('%Y-%m-%d')

                    return f"'{date_val}'"  # Enclose the date in quotes

                except ValueError:
                    print(f"ValueError: Unable to convert {value} to a date.")
                    return None
            else:
                # Return non-date values as is
                return value
        return None

    # Check if the columns are date type
    is_date = "_start_of_production" in lower_bound_col or "_start_of_production" in upper_bound_col


    lower_bound_value = format_value_for_query(lower_bound_value, is_date)
    upper_bound_value = format_value_for_query(upper_bound_value, is_date)

    # Logic for each range type
    if range_type == 'equal':
        return f"({pihs_vl_ps_col} == {lower_bound_value})" if not pd.isnull(lower_bound_value) else None
    elif range_type == 'less than':
        return f"({pihs_vl_ps_col} < {upper_bound_value})" if not pd.isnull(upper_bound_value) else None
    elif range_type == 'greater than':
        return f"({pihs_vl_ps_col} > {lower_bound_value})" if not pd.isnull(lower_bound_value) else None
    elif range_type == 'greater or equal':
        return f"({pihs_vl_ps_col} >= {lower_bound_value})" if not pd.isnull(lower_bound_value) else None
    elif range_type == 'less or equal':
        return f"({pihs_vl_ps_col} <= {upper_bound_value})" if not pd.isnull(upper_bound_value) else None
    elif range_type == 'exclusive-exclusive':
        return f"({pihs_vl_ps_col} > {lower_bound_value}) & ({pihs_vl_ps_col} < {upper_bound_value})" if not pd.isnull(lower_bound_value) and not pd.isnull(upper_bound_value) else None
    elif range_type == 'exclusive-inclusive':
        return f"({pihs_vl_ps_col} > {lower_bound_value}) & ({pihs_vl_ps_col} <= {upper_bound_value})" if not pd.isnull(lower_bound_value) and not pd.isnull(upper_bound_value) else None
    elif range_type == 'inclusive-exclusive':
        return f"({pihs_vl_ps_col} >= {lower_bound_value}) & ({pihs_vl_ps_col} < {upper_bound_value})" if not pd.isnull(lower_bound_value) and not pd.isnull(upper_bound_value) else None
    elif range_type == 'inclusive-inclusive':
        return f"({pihs_vl_ps_col} >= {lower_bound_value}) & ({pihs_vl_ps_col} <= {upper_bound_value})" if not pd.isnull(lower_bound_value) and not pd.isnull(upper_bound_value) else None
    elif range_type == 'non-inclusive gap':
        return f"(({pihs_vl_ps_col} < {lower_bound_value}) | ({pihs_vl_ps_col} > {upper_bound_value}))" if not pd.isnull(lower_bound_value) and not pd.isnull(upper_bound_value) else None

    return None

# Define the construct_query function
def construct_query(row, exact_match_columns, range_match_columns):
    query_parts = [f"{col} == '{row[col]}'" for col in exact_match_columns if pd.notnull(row[col])]

    for lower_bound_col, upper_bound_col, range_type_col, pihs_vl_ps_col in range_match_columns:
        range_query = handle_range_match(row, lower_bound_col, upper_bound_col, range_type_col, pihs_vl_ps_col)
        if range_query:
            query_parts.append(range_query)

    return ' & '.join(query_parts) if query_parts else None

# Initialize a DataFrame to store queries
query_log = pd.DataFrame(columns=['rule_number', 'query'])

# Additional columns from sd to append to merged DataFrame
additional_columns_sd = sd.columns[sd.columns.get_loc('nur_of_eaxle') + 1:].tolist()

# Initialize merged DataFrame with all rows from pihs_vl_ps and additional columns from sd
merged_columns = pihs_vl_ps.columns.tolist() + additional_columns_sd + ['pihs_vl_ps_sd_status']
merged_df = pihs_vl_ps.reindex(columns=merged_columns)
merged_df['pihs_vl_ps_sd_status'] = 'unmatched'

# Add a marker column in pihs_vl_ps to track matched pihs_vl_ps_sd_status
pihs_vl_ps['matched'] = False

# Iterate over each rule in sd and apply to pihs_vl_ps, logging queries
for index, row in sd.iterrows():
    rule_number = row['row']  # Assuming 'row' is a column in sd
    query = construct_query(row, exact_match_columns, range_match_columns)
    if query:
        query_log = query_log.append({'rule_number': rule_number, 'query': query}, ignore_index=True)
        print(query)
        matched_rows = pihs_vl_ps.query(query).index
        for idx in matched_rows:
            if not pihs_vl_ps.at[idx, 'matched']:
                merged_df.loc[idx, additional_columns_sd] = row[additional_columns_sd]
                merged_df.loc[idx, 'pihs_vl_ps_sd_status'] = row['row']
                pihs_vl_ps.at[idx, 'matched'] = True

# Filter merged_df by 'unmatched' pihs_vl_ps_sd_status
unmerged_df = merged_df[merged_df['pihs_vl_ps_sd_status'] == 'unmatched']

# Generate file name for the processed files
paris_tz = pytz.timezone('Europe/Paris')
current_time = datetime.now(paris_tz).strftime("%Y%m%d_%H%M%S")

# Save query_log to GCS bucket
query_log_file_name = f'pihs_vl_ps_sd_query_log_{current_time}.csv'
query_log_folder = '04_dictionary/07_pihs_vl_ps_sd_query_log'
save_to_bucket(query_log, bucket, query_log_folder, query_log_file_name)

# Save merged_df to GCS bucket
merged_df_file_name = f'pihs_vl_ps_sd_merged_{current_time}.csv'
merged_df_folder = '04_dictionary/04_pihs_vl_ps_sd_merged'
save_to_bucket(merged_df, bucket, merged_df_folder, merged_df_file_name)

# Save unmerged_df to GCS bucket
unmerged_df_file_name = f'pihs_vl_ps_sd_unmerged_{current_time}.csv'
unmerged_df_folder = '04_dictionary/05_pihs_vl_ps_sd_unmerged'
save_to_bucket(unmerged_df, bucket, unmerged_df_folder, unmerged_df_file_name)

# Save sorted_sd to GCS bucket
sorted_sd_file_name = f'pihs_vl_ps_sd_sorted_sd_{current_time}.csv'
sorted_sd_folder = '04_dictionary/06_pihs_vl_ps_sd_sorted_sd'
save_to_bucket(sorted_sd, bucket, sorted_sd_folder, sorted_sd_file_name)

# Print statements to confirm the process
print("\nQuery log:")
print(query_log.head())
print("\nSample of merged DataFrame:")
print(merged_df.head())
